⚠️ NOTE:
This notebook is for research and validation only.
Production logic lives in the `src/` directory.

In [1]:
# Project path setup
import sys
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\shubh\crypto-market-opportunity-engine")
sys.path.insert(0, str(PROJECT_ROOT))

print("✅ Project root added:", PROJECT_ROOT)


✅ Project root added: C:\Users\shubh\crypto-market-opportunity-engine


In [2]:
import pandas as pd
import numpy as np

from src.inference import load_model
import src.config as config


In [3]:
DATA_PROCESSED = r"C:\Users\shubh\crypto-market-opportunity-engine\data\processed\BTCUSDT_5m_2025_features.parquet"

df = pd.read_parquet(DATA_PROCESSED)

print("Shape:", df.shape)
print("Columns:", df.columns.tolist())


Shape: (105092, 26)
Columns: ['open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'log_return', 'volatility', 'rsi', 'rsi_14', 'bb_high', 'bb_low', 'ema_diff', 'vol_ratio', 'ema_diff_lag_1', 'ema_diff_lag_2', 'ema_diff_lag_3', 'rsi_14_lag_1', 'rsi_14_lag_2', 'vol_ratio_lag_1', 'trend', 'target']


In [4]:
# prepare x features only
FEATURE_COLS = config.FEATURE_COLS

X = df[FEATURE_COLS]

print("X shape:", X.shape)


X shape: (105092, 10)


In [5]:
# load trained model
MODEL_NAME = "random_forest"
VERSION = "1"

model = load_model(MODEL_NAME, VERSION)

print("✅ Model loaded:", MODEL_NAME, VERSION)


✅ Model loaded: random_forest 1


In [6]:
# generate prbalitites
df["prob_up"] = model.predict_proba(X)[:, 1]

df[["prob_up"]].head()


,prob_up
open_time,
2025-01-01 02:20:00,0.503679
2025-01-01 02:25:00,0.504466
2025-01-01 02:30:00,0.503574
2025-01-01 02:35:00,0.499005
2025-01-01 02:40:00,0.492561


In [7]:
# define signal logic
BUY_THRESHOLD = 0.60
SELL_THRESHOLD = 0.40

def generate_signal(p):
    if p >= BUY_THRESHOLD:
        return "BUY"
    elif p <= SELL_THRESHOLD:
        return "SELL"
    else:
        return "HOLD"

df["signal"] = df["prob_up"].apply(generate_signal)

df["signal"].value_counts(normalize=True)


signal
HOLD    0.999467
BUY     0.000438
SELL    0.000095
Name: proportion, dtype: float64

In [8]:
# signal sanity check
df[["prob_up", "signal"]].tail(10)


,prob_up,signal
open_time,,
2025-12-31 23:10:00,0.493308,HOLD
2025-12-31 23:15:00,0.490690,HOLD
2025-12-31 23:20:00,0.486662,HOLD
2025-12-31 23:25:00,0.488431,HOLD
2025-12-31 23:30:00,0.489571,HOLD
2025-12-31 23:35:00,0.492735,HOLD
2025-12-31 23:40:00,0.494017,HOLD
2025-12-31 23:45:00,0.492686,HOLD
2025-12-31 23:50:00,0.489541,HOLD


In [11]:
SIGNAL_PATH = r"C:\Users\shubh\crypto-market-opportunity-engine\data\processed\BTCUSDT_5m_signals.parquet"

df.to_parquet(SIGNAL_PATH)

print("✅ Signals saved to:", SIGNAL_PATH)


✅ Signals saved to: C:\Users\shubh\crypto-market-opportunity-engine\data\processed\BTCUSDT_5m_signals.parquet
